In [1]:
%load_ext autoreload
# %autoreload 2

In [2]:
import logging
from sys import stdout
from datetime import datetime
import numpy as np
import os

logging.basicConfig(
    level=logging.INFO,
    format="[%(asctime)s][%(levelname)s] %(message)s",
    stream=stdout,
    datefmt="%m-%d %H:%M:%S",
)

logging.info(f"{os.getlogin()}| {datetime.now()}")

[08-07 13:26:36][INFO] bravo| 2025-08-07 13:26:36.915996


In [3]:
# ####################################################################
def gauss_jordan(Ab: np.ndarray) -> np.ndarray:
    """Resuelve un sistema de ecuaciones lineales mediante el método de Gauss-Jordan.

    ## Parameters

    ``Ab``: matriz aumentada del sistema de ecuaciones lineales. Debe ser de tamaño n-by-(n+1), donde n es el número de incógnitas.

    ## Return

    ``solucion``: vector con la solución del sistema de ecuaciones lineales.

    """
    if not isinstance(Ab, np.ndarray):
        logging.debug("Convirtiendo A a numpy array.")
        Ab = np.array(Ab, dtype=float)
    assert Ab.shape[0] == Ab.shape[1] - 1, "La matriz A debe ser de tamaño n-by-(n+1)."
    n = Ab.shape[0]

    for i in range(0, n):  # loop por columna

        # --- encontrar pivote
        p = None  # default, first element
        for pi in range(i, n):
            if Ab[pi, i] == 0:
                # must be nonzero
                continue

            if p is None:
                # first nonzero element
                p = pi
                continue

            if abs(Ab[pi, i]) < abs(Ab[p, i]):
                p = pi

        if p is None:
            # no pivot found.
            raise ValueError("No existe solución única.")

        if p != i:
            # swap rows
            logging.debug(f"Intercambiando filas {i} y {p}")
            _aux = Ab[i, :].copy()
            Ab[i, :] = Ab[p, :].copy()
            Ab[p, :] = _aux

        # --- Eliminación: loop por fila
        for j in range(n):
            if i == j:
                continue
            m = Ab[j, i] / Ab[i, i]
            Ab[j, i:] = Ab[j, i:] - m * Ab[i, i:]

        logging.info(f"\n{Ab}")

    if Ab[n - 1, n - 1] == 0:
        raise ValueError("No existe solución única.")

    # --- Sustitución hacia atrás
    solucion = np.zeros(n)

    for i in range(n - 1, -1, -1):
        solucion[i] = Ab[i, -1] / Ab[i, i]

    return solucion

In [4]:
%autoreload 2
from src import matriz_aumentada

A = [
    [1, 2, 3, 4],
    [2, 5, 6, 7],
    [3, 6, 8, 9],
    [4, 7, 9, 10],
]
b = [1, -2, 3, 4]

Ab = matriz_aumentada(A, b)
gauss_jordan(Ab)

[08-07 13:26:42][INFO] 2025-08-07 13:26:42.806700
[08-07 13:26:42][INFO] bravo| 2025-08-07 13:26:42.808146
[08-07 13:26:42][INFO] 2025-08-07 13:26:42.809270
[08-07 13:26:42][INFO] bravo| 2025-08-07 13:26:42.810082
[08-07 13:26:42][INFO] 
[[ 1.  2.  3.  4.  1.]
 [ 0.  1.  0. -1. -4.]
 [ 0.  0. -1. -3.  0.]
 [ 0. -1. -3. -6.  0.]]
[08-07 13:26:42][INFO] 
[[ 1.  0.  3.  6.  9.]
 [ 0.  1.  0. -1. -4.]
 [ 0.  0. -1. -3.  0.]
 [ 0.  0. -3. -7. -4.]]
[08-07 13:26:42][INFO] 
[[ 1.  0.  0. -3.  9.]
 [ 0.  1.  0. -1. -4.]
 [ 0.  0. -1. -3.  0.]
 [ 0.  0.  0.  2. -4.]]
[08-07 13:26:42][INFO] 
[[ 1.  0.  0.  0.  3.]
 [ 0.  1.  0.  0. -6.]
 [ 0.  0. -1.  0. -6.]
 [ 0.  0.  0.  2. -4.]]


array([ 3., -6.,  6., -2.])

# Resolver

In [5]:
# ####################################################################
def inv_matrix(A: np.ndarray) -> np.ndarray:
    """Inversión de una matriz cuadrada mediante método de Gauss-Jordan.
    ## Parameters
    ``A``: matriz cuadrada de tamaño n x n.

    ## Return
    ``A_inv``: matriz inversa de A.
    """
    # COMPLETAR
    # Debe basarse en la función gauss_jordan
    pass

In [13]:
def inv_matrix(A: np.ndarray) -> np.ndarray:
    """Inversión de una matriz cuadrada mediante método de Gauss-Jordan.
    ## Parameters
    ``A``: matriz cuadrada de tamaño n x n.

    ## Return
    ``A_inv``: matriz inversa de A.
    """
    A = np.array(A, dtype=float)
    n = A.shape[0]

    # Crear matriz aumentada [A | I]
    I = np.eye(n)
    AI = np.hstack([A, I])

    # Aplicar Gauss-Jordan para obtener [I | A^(-1)]
    for i in range(n):
        # Buscar pivote
        pivot_row = i
        for j in range(i + 1, n):
            if abs(AI[j, i]) > abs(AI[pivot_row, i]):
                pivot_row = j

        # Intercambiar filas si es necesario
        if pivot_row != i:
            AI[[i, pivot_row]] = AI[[pivot_row, i]]

        # Verificar que el pivote no sea cero
        if abs(AI[i, i]) < 1e-10:
            raise ValueError("La matriz no es invertible")

        # Normalizar fila del pivote
        AI[i] = AI[i] / AI[i, i]

        # Eliminar columna
        for j in range(n):
            if i != j:
                AI[j] = AI[j] - AI[j, i] * AI[i]

    # Extraer la matriz inversa
    return AI[:, n:]


In [14]:
def verificar_inversa(A, A_inv):
    """Verifica que A * A_inv = I"""
    A = np.array(A)
    producto = np.dot(A, A_inv)
    identidad = np.eye(A.shape[0])
    return np.allclose(producto, identidad, atol=1e-10)

## Ejemplos
* Ejemplo 1

In [15]:
# La matriz A =
A = [
    [1, 2, 3, 4],
    [2, 5, 6, 7],
    [3, 6, 8, 9],
    [4, 7, 9, 10],
]
# tiene como inversa
# A_inv =[[ 0.5, -0.5, -1.5,  1.5],
#        [-0.5,  1.5, -1.5,  0.5],
#        [-1.5, -1.5,  3.5, -1.5],
#        [ 1.5,  0.5, -1.5,  0.5]]
inv_matrix(A)

array([[ 0.5, -0.5, -1.5,  1.5],
       [-0.5,  1.5, -1.5,  0.5],
       [-1.5, -1.5,  3.5, -1.5],
       [ 1.5,  0.5, -1.5,  0.5]])

* Ejemplo 2

In [16]:
# La matriz A =
A = [
    [4, 4, 5, 1],
    [3, 4, 2, 2],
    [2, 1, 4, 1],
    [3, 2, 5, 4],
]
# tiene como inversa
# A_inv =[[-34.,  31.,  52., -20.],
#         [ 19., -17., -29.,  11.],
#         [ 12., -11., -18.,   7.],
#         [  1.,  -1.,  -2.,   1.]]
inv_matrix(A)

array([[-34.,  31.,  52., -20.],
       [ 19., -17., -29.,  11.],
       [ 12., -11., -18.,   7.],
       [  1.,  -1.,  -2.,   1.]])

## Ejercicios

* Ejercicio 1

In [17]:
A = [[2, -3], [-1, 1]]
inv_matrix(A)

array([[-1., -3.],
       [-1., -2.]])

* Ejercicio 2

In [18]:
A = [
    [4, 0, 0, 5],
    [1, 0, 4, 0],
    [3, 4, 1, 3],
    [1, 3, 3, 0],
]
inv_matrix(A)

array([[-36.,  45.,  60., -80.],
       [  3.,  -4.,  -5.,   7.],
       [  9., -11., -15.,  20.],
       [ 29., -36., -48.,  64.]])

* Ejercicio 3

In [19]:
A = [
    [0, 0, 0, 0, 0, 0, 1, -1],
    [0, 1, -1, 1, 0, -1, 0, 1],
    [-1, -1, 0, 0, 2, 1, 0, 0],
    [-1, -1, -1, 1, 2, 0, 0, 1],
    [-1, 1, 1, 0, -1, -1, 0, 2],
    [0, 1, 0, 0, -1, -1, 0, 0],
    [1, -1, -1, 1, 2, 1, 0, 2],
    [2, 0, 0, 0, 0, 1, 2, 0],
]
inv_matrix(A)

array([[ 2., -1.,  0., -1.,  0.,  2.,  2., -1.],
       [ 0.,  1.,  1., -1.,  0.,  0.,  0.,  0.],
       [ 6., -1.,  0., -3.,  1.,  1.,  4., -3.],
       [ 6.,  1., -1., -3.,  1., -3.,  3., -3.],
       [ 2., -1.,  1., -1.,  0.,  3.,  2., -1.],
       [-2.,  2., -0.,  0., -0., -4., -2.,  1.],
       [-1., -0., -0.,  1., -0., -0., -1.,  1.],
       [-2., -0., -0.,  1., -0., -0., -1.,  1.]])

* Ejercicio 4

In [20]:
A = [
    [1, 0, 0, 0, -1, 0, 0, -1, 1, -1],
    [1, 1, 0, -1, -1, 1, 0, 0, 1, -1],
    [-1, 0, -1, 0, 0, 0, -1, 1, 0, 0],
    [0, 0, -1, 0, -1, -1, 1, 0, 1, 0],
    [-1, 0, 0, -1, 1, 1, 1, 1, 0, -1],
    [1, 0, 0, 1, -1, -1, -1, 1, -1, 0],
    [1, 1, 1, 0, 1, 0, -1, -1, -1, 1],
    [1, 1, 1, 1, 0, 0, 1, 1, 0, 0],
    [1, 1, 1, 1, 1, 0, -1, -1, 0, 0],
    [0, 0, -1, -1, -1, 0, 1, 1, 1, -1],
]
inv_matrix(A)

array([[ 14.,  -8.,   9.,  -4.,   0.,  -4.,   9.,   7.,  -8.,   3.],
       [ -2.,   2.,  -1.,   2.,   1.,   1.,  -1.,  -1.,   1.,  -2.],
       [-27.,  14., -18.,   5.,  -2.,   7., -17., -13.,  16.,  -2.],
       [ 12.,  -6.,   8.,  -2.,   1.,  -3.,   7.,   6.,  -7.,   0.],
       [  6.,  -4.,   4.,  -2.,   0.,  -2.,   4.,   3.,  -3.,   2.],
       [ 18.,  -9.,  12.,  -4.,   1.,  -5.,  11.,   9., -11.,   1.],
       [  8.,  -4.,   5.,  -1.,   1.,  -2.,   5.,   4.,  -5.,   0.],
       [ -5.,   2.,  -3.,   0.,  -1.,   1.,  -3.,  -2.,   3.,   1.],
       [-11.,   5.,  -7.,   1.,  -2.,   2.,  -7.,  -5.,   7.,   1.],
       [  1.,  -1.,   1.,  -1.,  -1.,  -1.,   1.,   1.,  -1.,   1.]])

In [21]:
import numpy as np
from src.linear_sist_methods import gauss_jordan, matriz_aumentada

def inv_matrix(A):
    """
    Calcula A^{-1} resolviendo A x = e_i para i=0..n-1
    con el método de Gauss–Jordan (función gauss_jordan sobre matriz aumentada).
    """
    A = np.array(A, dtype=float)
    n = A.shape[0]
    inv = np.zeros((n, n), dtype=float)
    for i in range(n):
        # vector columna de la identidad
        b = np.zeros(n, dtype=float)
        b[i] = 1.0
        # construir matriz [A | b]
        Ab = matriz_aumentada(A, b)
        # resolver A x = b
        x = gauss_jordan(Ab)
        inv[:, i] = x.flatten()
    return inv

# — Ejercicio 1 —
A1 = [[2, -3],
      [-1, 1]]
A1_inv = inv_matrix(A1)
print("Ejercicio 1: A^{-1} =")
print(A1_inv)

# — Ejercicio 2 —
A2 = [
    [ 4, 0, 0, 5],
    [ 1, 0, 4, 0],
    [ 3, 4, 1, 3],
    [ 1, 3, 3, 0],
]
A2_inv = inv_matrix(A2)
print("\nEjercicio 2: A^{-1} =")
print(A2_inv)


[08-07 13:30:41][INFO] Intercambiando filas 0 y 1.
[08-07 13:30:41][INFO] 
[[-1.  1.  0.]
 [ 0. -1.  1.]]
[08-07 13:30:41][INFO] 
[[-1.  0.  1.]
 [ 0. -1.  1.]]
[08-07 13:30:41][INFO] Intercambiando filas 0 y 1.
[08-07 13:30:41][INFO] 
[[-1.  1.  1.]
 [ 0. -1.  2.]]
[08-07 13:30:41][INFO] 
[[-1.  0.  3.]
 [ 0. -1.  2.]]
Ejercicio 1: A^{-1} =
[[-1. -3.]
 [-1. -2.]]
[08-07 13:30:41][INFO] Intercambiando filas 0 y 1.
[08-07 13:30:41][INFO] 
[[  1.   0.   4.   0.   0.]
 [  0.   0. -16.   5.   1.]
 [  0.   4. -11.   3.   0.]
 [  0.   3.  -1.   0.   0.]]
[08-07 13:30:41][INFO] Intercambiando filas 1 y 3.
[08-07 13:30:41][INFO] 
[[  1.           0.           4.           0.           0.        ]
 [  0.           3.          -1.           0.           0.        ]
 [  0.           0.          -9.66666667   3.           0.        ]
 [  0.           0.         -16.           5.           1.        ]]
[08-07 13:30:41][INFO] 
[[ 1.          0.          0.          1.24137931  0.        ]
 [ 0.     